# Reflection

In the context of LLM agent building, reflection refers to the process of
prompting an LLM to observe its past steps (along with potential observations
from tools/the environment) to assess the quality of the chosen actions. This is
then used downstream for things like re-planning, search, or evaluation.

![Reflection](./img/reflection.png)

This notebook demonstrates a very simple form of reflection in LangGraph.

#### Prerequisites

We will be using a basic agent with a search tool here.

## Setup

### Load env vars

Add a `.env` variable in the root of the repo folder with your variables.

In [1]:
import "npm:dotenv/config";

[Module: null prototype] { default: {} }

## Generate

For our example, we will create a "5 paragraph essay" generator. First, create
the generator:

In [23]:
import { ChatOpenAI } from "npm:@langchain/openai";
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "npm:@langchain/core/prompts";

const prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    `You are an essay assistant tasked with writing excellent 5-paragraph essays.
Generate the best essay possible for the user's request.
If the user provides critique, respond with a revised version of your previous attempts.`,
  ],
  new MessagesPlaceholder("messages"),
]);

const llm = new ChatOpenAI({
  openAIApiKey: "sk-mx15Qk5ghd20jeHusl",
  stop: ["assistant\n"],
  stream: true,
  configuration: {
    baseURL: "http://47.189.140.220:5000/v1",
  },
});
const essayGenerationChain = prompt.pipe(llm);

In [24]:
import { AIMessage, BaseMessage, HumanMessage } from "npm:@langchain/core/messages";

let essay = "";
const request = new HumanMessage({
  content:
    "Write an essay on why the little prince is relevant in modern childhood",
});

for await (
  const chunk of await essayGenerationChain.stream({ messages: [request] })
) {
  console.log(chunk.content);
  essay += chunk.content;
}


The
 Little
 Prince
,
 written
 by
 Antoine
 de
 Saint
-
Ex
up
éry
 in
 
194
3
,
 h
as remained
 
a timeless
 and
 poignant
 tale
 that
 continues
 to
 capt
ivate
 children
 and
 adults
 alike
.
 Despite
 being
 written
 over
 
75
 years
 ago
,
 this
 beloved
 nov
ell
a remains
 remarkably
 relevant
 in
 modern
 childhood
,
 offering
 valuable
 lessons
 and
 insights
 that
 are
 just
 
as applicable
 today
 
as they
 were
 when
 the
 book
 w
as first
 published
.


One
 of
 the
 primary
 reasons
 The
 Little
 Prince
 remains
 relevant
 is
 its
 exploration
 of
 the
 human
 condition
,
 particularly
 in
 the
 context
 of
 childhood
.
 The
 story
's
 protagonist
,
 
a young
 prince
 from
 
a distant
 asteroid
,
 gr
app
les
 with
 complex
 emotions
 and
 moral
 dile
mm
as that
 are
 universally
 rel
atable
,
 from
 the
 struggle
 to
 understand
 adulthood
 to
 the
 pain
 of
 saying
 goodbye
 to
 loved
 ones
.
 These
 themes
 are
 just
 
as pertinent
 today
,
 
as children
 navigate
 the


"The Little Prince, written by Antoine de Saint-Exupéry in 1943, has remained a timeless and poignant"... 3095 more characters

In [ ]:
essay

In [10]:
messages

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Write an essay on why the little prince is relevant in modern childhood",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Write an essay on why the little prince is relevant in modern childhood",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
]

### Reflect

In [19]:
const reflectionPrompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    `You are a teacher grading an essay submission.
Generate critique and recommendations for the user's submission.
Provide detailed recommendations, including requests for length, depth, style, etc.`,
  ],
  new MessagesPlaceholder("messages"),
]);
const reflect = reflectionPrompt.pipe(llm);

In [20]:
let reflection = "";

for await (
  const chunk of await reflect.stream({
    messages: [request, new HumanMessage({ content: essay })],
  })
) {
  console.log(chunk.content);
  reflection += chunk.content;
}


**
Grade
:
 B
+
**


**
Strength
s
:
**


*
 You
've
 provided
 
a clear
 and
 concise
 thesis
 statement
 that
 sets
 the
 tone
 for
 the
 rest
 of
 the
 essay
.

*
 Your
 introduction
 effectively
 establishes
 the
 relevance
 of
 The
 Little
 Prince
 in
 modern
 childhood
,
 making
 the
 reader
 curious
 to
 learn
 more
.

*
 You
've
 identified
 
a specific
 aspect
 of
 the
 book
 (
expl
oration
 of
 complex
 emotions
 and
 relationships
)
 
as 
a key
 reason
 for
 its
 relevance
,
 which
 shows
 you
've
 engaged
 with
 the
 material
.


**
Weak
ness
es
 and
 Recommendations
:
**


1



"**Grade: B+**\n" +
  "\n" +
  "**Strengths:**\n" +
  "\n" +
  "* You've provided a clear and concise thesis statement that sets the "... 395 more characters

### Repeat

And... that's all there is too it! You can repeat in a loop for a fixed number
of steps, or use an LLM (or other check) to decide when the finished product is
good enough.

In [22]:
console.log(reflection)

**Grade: B+**

**Strengths:**

* You've provided a clear and concise thesis statement that sets the tone for the rest of the essay.
* Your introduction effectively establishes the relevance of The Little Prince in modern childhood, making the reader curious to learn more.
* You've identified a specific aspect of the book (exploration of complex emotions and relationships) as a key reason for its relevance, which shows you've engaged with the material.

**Weaknesses and Recommendations:**

1


In [6]:
const stream = await essayGenerationChain.stream({
  messages: [
    request,
    new AIMessage({ content: essay }),
    new HumanMessage({ content: reflection }),
  ],
});
for await (const chunk of stream) {
  console.log(chunk.content);
}


Dear [
Your Name],

Thank you for your thought
ful and constructive feedback on my essay about the
 relevance of The Little Prince in modern childhood.
 I appreciate the time you took to provide
 detailed suggestions for improvement, and I am eager to
 revise my work to better convey my ideas and
 engage the reader.

1. Essay
 Length: I will expand upon my ideas and provide
 more specific examples from the text to create a more
 comprehensive analysis. This will allow me to delve
 deeper into the themes and messages of the novella
 and strengthen my arguments.
2. Depth
: I will further explore how the themes of l
oneliness, responsibility, and the search for meaning
 are reflected in the modern world and how they impact
 children's lives. By providing more context and
 real-world examples, I aim to solidify
 my arguments and demonstrate the relevance of these themes
 in contemporary society.
3. Style: I
 will incorporate more varied sentence structures and vocabulary to
 make my essay

## Define graph

Now that we've shown each step in isolation, we can wire it up in a graph.

In [7]:
import { END, MessageGraph } from "@langchain/langgraph";

const generationNode = async (messages: BaseMessage[]) => {
  return [
    await essayGenerationChain.invoke({ messages }),
  ];
};

const reflectionNode = async (messages: BaseMessage[]) => {
  // Other messages we need to adjust
  const clsMap: { [key: string]: new (content: string) => BaseMessage } = {
    "ai": HumanMessage,
    "human": AIMessage,
  };
  // First message is the original user request. We hold it the same for all nodes
  const translated = [
    messages[0],
    ...messages.slice(1).map((msg) => new clsMap[msg._getType()](msg.content)),
  ];
  const res = await reflect.invoke({ "messages": translated });
  // We treat the output of this as human feedback for the generator
  return [new HumanMessage(res.content)];
};

// Define the graph
const workflow = new MessageGraph();
workflow.addNode("generate", generationNode);
workflow.addNode("reflect", reflectionNode);
workflow.setEntryPoint("generate");

const shouldContinue = (messages: BaseMessage[]) => {
  if (messages.length > 6) {
    // End state after 3 iterations
    return END;
  }
  return "reflect";
};

workflow.addConditionalEdges("generate", shouldContinue);
workflow.addEdge("reflect", "generate");

const app = workflow.compile();

In [8]:
let finalRes;

const stream = await app.stream([
  new HumanMessage({
    content:
      "Generate an essay on the topicality of The Little Prince and its message in modern life",
  }),
]);

for await (const event of stream) {
  finalRes = event;
  for (const [key, value] of Object.entries(event)) {
    console.log(`Event: ${key}`);
    // Uncomment to see the result of each step.
    // console.log(value.map((msg) => msg.content).join("\n"));
    console.log("\n------\n");
  }
}

Event: generate

------

Event: reflect

------

Event: generate

------

Event: reflect

------

Event: generate

------

Event: reflect

------

Event: generate

------

Event: __end__

------



In [9]:
console.log(
  finalRes[END].map((msg) => msg.content).join(
    "\n\n\n------------------\n\n\n",
  ),
);

Generate an essay on the topicality of The Little Prince and its message in modern life


------------------


Title: The Little Prince: A Timeless Message in Modern Life

Introduction:
Antoine de Saint-Exupéry's The Little Prince is a classic novella that has captured the hearts of millions of readers worldwide since its publication in 1943. The story, which follows a young prince traveling through various planets, carries a profound message about human nature, relationships, and the pursuit of meaning in life. Despite being over seven decades old, The Little Prince's message remains highly relevant in today's fast-paced, technology-driven modern world.

Body Paragraph 1 - The Relevance of The Little Prince's Themes:
The Little Prince explores themes such as love, friendship, responsibility, and the importance of looking beneath the surface to understand the true nature of things. These themes resonate with readers of all ages and backgrounds, as they touch upon fundamental aspects of

> #### See the LangSmith trace [here](https://smith.langchain.com/public/4e3c3424-e2fc-44c8-baf4-a7da22bae210/r)